In [1]:
from dataset import *
from train import *


In [2]:
import os
import pandas as pd

import torch
import torch.optim as optim

import collections

In [3]:
# Set parameters
EuroSat_Type = 'ALL'    
target_country = 'France'

lr = 0.01               # learn_rate
milestones = [50,75,90] # multistep scheduler
epochs = 100            # no of epochs

output_path = "./" + target_country

## Download data

In [4]:
# raw data
if EuroSat_Type == 'RGB':
  data_folder = '/content/sample_data/'
  #root = os.path.join(data_folder, '2750/')
  root = '2750/'
  download_ON = os.path.exists(root)

  if not download_ON:
    # This can be long...
    #os.chdir(data_folder)
    os.system('wget http://madm.dfki.de/files/sentinel/EuroSAT.zip') #Just RGB Bands
    !unzip EuroSAT.zip
    download_ON = True
elif EuroSat_Type == 'ALL':
    root = 'ds/images/remote_sensing/otherDatasets/sentinel_2/tif/'
    download_ON = os.path.exists(root)
    if not download_ON:
      os.system('wget http://madm.dfki.de/files/sentinel/EuroSATallBands.zip') #All bands
      !unzip EuroSATallBands.zip
      download_ON = True
    
data = torchvision.datasets.DatasetFolder(root=root,loader = iloader, transform=None, extensions = 'tif')

In [5]:
# Metadata
geo_df = pd.read_csv("metadata.csv")
geo_dict = geo_df.to_dict()
countries = list(set(geo_dict["country"].values()))
id_countries = dict.fromkeys(countries)
for k in id_countries.keys():
    id_countries[k] = [v for (i, v) in enumerate(geo_dict["id"]) if geo_dict["country"][i] == k]

In [6]:
# source - target split
id_target = id_countries[target_country]
id_train = random.sample(id_target, 320)
id_test = list(set(id_target) - set(id_train))[0:160]

loader_target_train = torch.utils.data.DataLoader(torch.utils.data.Subset(data, id_train), 
                                                  batch_size= 16, shuffle=False, num_workers=0)
loader_target_test = torch.utils.data.DataLoader(torch.utils.data.Subset(data, id_test), 
                                                  batch_size= 16, shuffle=False, num_workers=0)

id_random_source = random.sample(list(geo_dict["id"].values()),
                                len(loader_target_train.dataset))
loader_random_source = torch.utils.data.DataLoader(torch.utils.data.Subset(data, id_random_source), 
                                                  batch_size= 16, shuffle=False, num_workers=0)

In [8]:
label_dict = dict.fromkeys(id_countries.keys())
labels = dict.fromkeys(id_countries.keys())
for key in id_countries.keys():
    id_target = id_countries[key]
    labels_ = [data.samples[i][1] for i in id_target]
    labels[key] = labels_
    label_dict[key] = list(collections.Counter(labels_).keys())
    print(key, collections.Counter(labels_))

nan Counter()
Lietuva Counter({1: 18, 9: 17, 3: 16, 8: 16, 7: 11, 4: 10, 5: 2})
Slovensko Counter({1: 226, 0: 60, 4: 34, 8: 30, 3: 18, 7: 17, 6: 13, 9: 13, 5: 3})
Lëtzebuerg Counter({1: 33, 8: 17, 3: 9, 4: 2, 5: 2, 7: 1})
Бългaрия Counter({1: 233, 0: 142, 9: 81, 7: 67, 4: 50, 6: 31, 3: 26, 5: 16, 2: 11, 8: 10})
Österreich Counter({2: 313, 8: 230, 9: 206, 1: 132, 6: 115, 3: 70, 0: 47, 7: 33, 4: 21, 5: 7})
Κύπρος - Kıbrıs Counter({2: 380, 6: 21, 7: 10, 4: 6, 3: 4})
United Kingdom Counter({5: 990, 2: 787, 7: 351, 4: 334, 3: 207, 9: 184, 8: 79, 0: 42})
Polska Counter({1: 420, 9: 242, 0: 219, 8: 138, 3: 81, 7: 80, 4: 75, 5: 16})
België / Belgique / Belgien Counter({3: 69, 7: 53, 4: 52, 8: 37, 1: 33, 9: 21, 0: 1, 5: 1})
España Counter({2: 704, 6: 451, 3: 298, 0: 267, 4: 189, 7: 153, 1: 97, 8: 72, 9: 63, 5: 10})
Eesti Counter({9: 22, 3: 16, 4: 16, 5: 11, 7: 10, 1: 7})
Danmark Counter({7: 202, 9: 100, 3: 61, 4: 54, 2: 21, 0: 16, 8: 5, 5: 2})
Latvija Counter({8: 94, 5: 26, 9: 8, 3: 6, 4: 5, 0: 